In [ ]:
!pip install openai -q

from getpass import getpass
import openai

OPENAI_API_KEY = getpass('OpenAI API key: ')
openai.api_key = OPENAI_API_KEY

!pip install langchain -q
!pip install faiss-gpu -q
!pip install adapter-transformers -q
!pip install spacy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.8 MB/s eta 0:00:00
OpenAI API key: ··········
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.0 MB/s eta 0:00:00


# Params

In [ ]:
# Input files from last stages
#SURVEY_TOPIC_QUERY = 'text neural information retrieval'
#file_sections_structure = 'sections-2015-2023.pkl'
#file_papers_contents = 'papers_contents-2015-2023_grobid.pkl'

SURVEY_TOPIC_QUERY = 'neural information retrieval'
file_sections_structure = 'sections-survey-ref.pkl'
file_papers_contents = 'papers_contents-survey-ref_grobid.pkl'

PARAMS = {
  'batch_size': 32,
  'sentences_in_chunk': 7,
  'sentences_overlap': 2,
  'embeddings': 'specter', # possible values: 'specter' and 'splade',
  'truncate_chunks_at_n_chars': 5000, # before send to GPT
  'n_chunks_returned_by_vector_db': 20,
  'n_chunks_to_use_as_reference': 10,
  'gpt_model_name_reranking': 'gpt-3.5-turbo-0613',
  'gpt_model_name_write_text': 'gpt-3.5-turbo-0613'
}


DEBUG = True # Log intermediate results

In [ ]:
import pickle

with open(file_sections_structure, "rb") as f:
   sections = pickle.load(f)

with open(file_papers_contents, "rb") as f:
   all_papers_contents = pickle.load(f)

In [ ]:
temp_ids_papers = list(all_papers_contents.keys())
print(all_papers_contents[temp_ids_papers[0]].keys())
for idx in range(5):
  print(all_papers_contents[temp_ids_papers[idx]]['title'])
  print(all_papers_contents[temp_ids_papers[idx]]['text'][0:200])
  if 'text_grobid' in all_papers_contents[temp_ids_papers[idx]]:
    print(all_papers_contents[temp_ids_papers[idx]]['text_grobid'][0:200])
  print('.'*50)

dict_keys(['metadata', 'title', 'abstract', 'text', 'grobid_text', 'text_grobid'])
Pretrained Transformers for Text Ranking: BERT and Beyond
Introduction 4 1.1 Text Ranking Problems . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 6 1.2 A Brief History . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1

..................................................
Neural Approaches to Conversional Information Retrieval
introduction to CIR. We hope that this book will prove a valuable resource for students, researchers, and software developers. This manuscript is a working draft. Comments are welcome. arXiv:2201.0517

..................................................
Conversational Information Seeking - An Introduction to Conversational Search, Recommendation, and Question Answering
Introduction to Conversational Search, Recommendation, and Question Answering Suggested Citation: Hamed Zamani, Johanne R. Trippas, Jeff Dalton and Filip Radlin￾ski (2023), “Conve

# Classes for Specter and SPLADE embeddings

In [ ]:
# Based on https://github.com/hwchase17/langchain/blob/4379bd4cbb8482e70d8936f747abd5ae7663f977/langchain/embeddings/huggingface.py#L16

from transformers import AutoAdapterModel, AutoTokenizer, AutoModel
from torch import cuda, bfloat16
import transformers
import torch
from tqdm.auto import tqdm
import numpy as np

from pydantic import BaseModel, Extra, Field
from langchain.embeddings.base import Embeddings
from typing import Any, Dict, List, Optional

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

class SpecterEmbeddings(BaseModel, Embeddings):

  """Key word arguments to pass to the model."""
  encode_kwargs: Dict[str, Any] = Field(default_factory=dict)

  def __init__(self, **kwargs: Any):
    super().__init__(**kwargs)

    self.tokenizer = AutoTokenizer.from_pretrained('allenai/specter2')
    self.model = AutoModel.from_pretrained('allenai/specter2')

    self.model.load_adapter("allenai/specter2_proximity", source="hf", load_as="specter2_proximity", set_active=False)
    self.model.load_adapter("allenai/specter2_adhoc_query", source="hf", load_as="adhoc_query", set_active=False)

    self.device = device
    self.model.eval()
    self.model.to(self.device)

  @torch.no_grad()
  def embed_documents(self, texts: List[str]) -> List[List[float]]:
    """Compute doc embeddings using a HuggingFace transformer model.

    Args:
        texts: The list of texts to embed.

    Returns:
        List of embeddings, one for each text.
    """

    self.model.set_active_adapters(None)
    self.model.set_active_adapters("specter2_proximity")

    all_embeddings = []

    batch_size = 32
    show_progress_bar = True

    if 'batch_size' in self.encode_kwargs:
      batch_size = self.encode_kwargs['batch_size']
    if 'show_progress_bar' in self.encode_kwargs:
      show_progress_bar = self.encode_kwargs['show_progress_bar']

    # sort text for less padding
    length_sorted_idx = np.argsort([-len(sen) for sen in texts])
    texts_sorted = [texts[idx] for idx in length_sorted_idx]

    for start_index in tqdm(range(0, len(texts_sorted), batch_size), desc="Batches", disable=not show_progress_bar):
      texts_batch = texts_sorted[start_index:start_index+batch_size]

      inputs = self.tokenizer(texts_batch, padding=True, truncation=True,
                              return_tensors="pt", return_token_type_ids=False, max_length=512)

      output = self.model(**inputs.to(self.device))
      # take the first token in the batch as the embedding
      embeddings = output.last_hidden_state[:, 0, :]

      all_embeddings.extend(embeddings.tolist())

    return all_embeddings

  class Config:
      """Configuration for this pydantic object."""

      extra = Extra.allow

  @torch.no_grad()
  def embed_query(self, text: str) -> List[float]:
    """Compute query embeddings using a HuggingFace transformer model.

    Args:
        text: The text to embed.

    Returns:
        Embeddings for the text.
    """
    self.model.set_active_adapters(None)
    self.model.set_active_adapters("adhoc_query")

    inputs = self.tokenizer(text, padding=True, truncation=True,
                            return_tensors="pt", return_token_type_ids=False, max_length=512)

    output = self.model(**inputs.to(self.device))
    # take the first token in the batch as the embedding
    embeddings = output.last_hidden_state[:, 0, :]

    return embeddings.squeeze(0).tolist()

In [ ]:
# Based on https://github.com/hwchase17/langchain/blob/4379bd4cbb8482e70d8936f747abd5ae7663f977/langchain/embeddings/huggingface.py#L16

from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import cuda
import torch
from tqdm.auto import tqdm
import numpy as np
from pydantic import BaseModel, Extra, Field
from langchain.embeddings.base import Embeddings
from typing import Any, Dict, List

class SpladeEmbeddings(BaseModel, Embeddings):

    """Key word arguments to pass to the model."""
    encode_kwargs: Dict[str, Any] = Field(default_factory=dict)

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)

        model = "naver/splade-cocondenser-ensembledistil"
        self.device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.model = AutoModelForMaskedLM.from_pretrained(model).to(self.device)
        self.model.eval()


    @torch.no_grad()
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a HuggingFace transformer model.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """

        all_embeddings = []

        batch_size = 32
        show_progress_bar = True

        if 'batch_size' in self.encode_kwargs:
            batch_size = self.encode_kwargs['batch_size']
        if 'show_progress_bar' in self.encode_kwargs:
            show_progress_bar = self.encode_kwargs['show_progress_bar']

        # sort text for less padding
        length_sorted_idx = np.argsort([-len(sen) for sen in texts])
        texts_sorted = [texts[idx] for idx in length_sorted_idx]

        for start_index in tqdm(range(0, len(texts_sorted), batch_size),
                                desc="Embedding Docs", disable=not show_progress_bar,
                                leave=False):
            texts_batch = texts_sorted[start_index:start_index+batch_size]

            inputs = self.tokenizer(
                texts_batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            )

            logits = self.model(**inputs.to(self.device)).logits

            inter = torch.log1p(torch.relu(logits))
            token_max = torch.max(inter, dim=1)[0].tolist()

            all_embeddings.extend(token_max)

        return all_embeddings

    class Config:
      """Configuration for this pydantic object."""

      extra = Extra.allow

    @torch.no_grad()
    def embed_query(self, text: str) -> List[float]:
        """Compute query embeddings using a HuggingFace transformer model.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """

        return self.embed_documents([text])[0]

In [ ]:
if DEBUG:
  print(f"Embeddings: {PARAMS['embeddings']}")

batch_size_embeddings = 1 if device == 'cpu' else PARAMS['batch_size']

if PARAMS['embeddings'] == 'specter':
  embeddings = SpecterEmbeddings(encode_kwargs={'batch_size': batch_size_embeddings})
else:
  embeddings = SpladeEmbeddings(encode_kwargs={'batch_size': batch_size_embeddings})

Embeddings: specter


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

# SentenceSplitter

In [ ]:
from typing import Any, List, Optional
from transformers import AutoTokenizer
from langchain.docstore.document import Document
import copy
import spacy

class SentenceTextSplitter():
  def __init__(self, sentences_in_chunk=7, sentences_overlap=2, pipeline: str = "en_core_web_sm"):
    self._spacy_tokenizer = spacy.load(pipeline)
    self._sentences_in_chunk = sentences_in_chunk
    self._sentences_overlap = sentences_overlap

  def split_documents(self, docs: List[Document]) -> List[Document]:
    documents = []
    for i, doc in tqdm(enumerate(docs), desc="Splitting", total=len(docs)):
      for chunk in self.split_text(doc.page_content):
        new_doc = Document(
            page_content=chunk, metadata=copy.deepcopy(doc.metadata)
        )
        documents.append(new_doc)
    return documents

  def split_text(self, text: str) -> List[str]:
    self._spacy_tokenizer.max_length = len(text) + 100
    sentences_in_chunk = self._sentences_in_chunk
    sentences_overlap = self._sentences_overlap
    sentences = (str(s) for s in self._spacy_tokenizer(text).sents)

    chunks = []
    chunk = []

    for sentence in sentences:
      chunk.append(sentence)

      whole_text = ' '.join(chunk)

      # if there are at least min_sentences_in_chunk sentences in chunk and there are more tokens than token_chunk_limit
      if len(chunk) >= sentences_in_chunk:
        chunks.append(whole_text)
        chunk = chunk[-sentences_overlap:]

    if chunk is not None:
      chunks.append(' '.join(chunk))

    return chunks


text_splitter = SentenceTextSplitter(sentences_in_chunk=PARAMS['sentences_in_chunk'], sentences_overlap=PARAMS['sentences_overlap'])

# Function to create a vector database

In [ ]:
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS

def get_paper_contents(paper):
  print(paper)
  paperId = paper['paperId']
  # Check if the text extracted from grobid exists
  if paperId not in all_papers_contents:
    print(f'Paper {paperId} not found')
    return ''

  contents = all_papers_contents[paperId].get('text_grobid', '')
  # If id doesn't, get the text extracted from pdfium
  if contents != '':
    if DEBUG:
      print(f'Text from paper {paperId} extracted using Grobid')
    return contents
  else:
    return all_papers_contents[paperId].get('text', '')

def prepare_splitted_document_to_be_stored(splitted_documents):
  # Changes the contents to {TITLE}  [SEP] {chunk} if specter
  # If splade, consider the original chunk of text
  # In both cases, change [SEP] to {SEP} in the title and chunk
  for doc in splitted_documents:
    if PARAMS['embeddings'] == 'specter':
      new_page_content = f"{doc.metadata['title'][:200].replace('[SEP]', '{sep}')} [SEP] {doc.page_content.replace('[SEP]', '{sep}')}"
    else:
      new_page_content = f"{doc.page_content.replace('[SEP]', '{sep}')}"
    doc.page_content = new_page_content

def create_vector_db(papers):
  # Represent the paper contents as a Document to be split
  papers_as_documents = []
  for paper in papers:
    contents = get_paper_contents(paper)
    metadata = paper
    if contents != '':
      papers_as_documents.append(Document(page_content=contents, metadata=metadata))

  # Split the papers contents
  if DEBUG:
    print('Splitting documents')
  splitted_documents = text_splitter.split_documents(papers_as_documents)

  # Prepare documents contents to be indexed in the vector db
  prepare_splitted_document_to_be_stored(splitted_documents)

  # Create the vector db
  if DEBUG:
    print('Indexing documents')
  db = FAISS.from_documents(splitted_documents, embeddings)

  return db

cached_vector_db = {}

def get_cached_vector_db(full_section_title, papers):
  if full_section_title not in cached_vector_db:
    if DEBUG:
      print(f'The database for section {full_section_title} is not in cache. Creating...')
    cached_vector_db[full_section_title] = create_vector_db(papers)

  if DEBUG:
    print('Retrieving database from cache')

  return cached_vector_db[full_section_title]

# Function to query a vector database

In [ ]:
def search_in_vector_db(db, query, n_chunks_to_return_by_vector_db):
  return db.similarity_search(query, n_chunks_to_return_by_vector_db)

# Function to rerank a bunch of chunks of text based on the title of a section

In [ ]:
import re

def remove_citation(text):
  # "Aqui vai uma [1] citação. Será legal isso? Ou não? [2]""
  patternIEEE = r"\[\d+\]"
  # "This method (Doe, J., 2020) has been shown to outperform previously discussed methods (Smith, J. et al., 2014) and while it has its draw-backs, it is clear that the benefits outweigh the disadvantages (Jones, A. & Karver, B., 2009, Lubber, H. et al., 2013)."
  patternAPA1 = r"\s\([A-Z][a-z]+,\s[A-Z][a-z]?\.[^\)]*,\s\d{4}\)"

  new_text = re.sub(patternIEEE, '', text)
  new_text = re.sub(patternAPA1, '', new_text)

  return new_text


In [ ]:
from langchain.prompts import (ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.schema import (AIMessage, HumanMessage, SystemMessage)

# SYSTEM MESSAGE
rerank_system_template = "You are a renowned scientist who is writing the section '{section_title}' of a survey about '{survey_topic}'."
rerank_system_message_prompt = SystemMessagePromptTemplate.from_template(rerank_system_template)

# FIRST HUMAN MESSAGE - EXPLAINING THE TASK
rerank_human_template_task = """\
I've found a text excerpt from a scientifc article that might be useful for the section '{section_title}' of your survey about '{survey_topic}'.\n\
Your task is to generate a score for it ranging from 0 to 5 indicating its importance to the section that you are writing.\n\
The score of a text written in a language other than English must be 0.\n\
If the text contains only metadata of an article (title, authors, venue, etc), i.e. if the text belongs to the references session, the score should be 0.\n\
You should also explain why you choose this score.\n\
Your answer MUST be enclosed in a RFC8259 compliant JSON object with two properties, "score" and "reasoning", \
containing the score and the reasoning for it. Remember to enclose the value of the reasoning property in quotes.\n\
Do not answer with anything besides the JSON object. Do not insert any text before or after the RFC8259 compliant JSON object.\n\
Use the following format to answer (write the reasoning property first and than write the score): \n\n\

```\n\
{{\n\
  "reasoning": {{REASONING}}\n\
  "score": {{SCORE}},\n\
}}\n\
```
"""
rerank_human_message_prompt_task = HumanMessagePromptTemplate.from_template(rerank_human_template_task)

# FIRST AI ANSWER - AGREEING
rerank_ai_message_prompt_yes = AIMessagePromptTemplate.from_template('Sure, send me the text I will give you what you need. I will answer with only a JSON object containg the score and the reasoning.')

# SECOND HUMAN MESSAGE - ABSTRACT AND TITLE
rerank_human_message_prompt_text = HumanMessagePromptTemplate.from_template('Text: {text}')

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
import ast

def get_prepared_text_from_document_chunk(doc):
  text = doc.page_content
  # If the embeddings of Specter was used, the page_content has the title
  if PARAMS['embeddings'] == 'specter':
    text = text[len(doc.metadata['title'])+7:] # page_content contains title [SEP] content. Remove title and [SEP]
  text = remove_citation(text) # Remove citations in the text
  return text

def parse_result_to_get_score(gpt_result):
  open_curly_brackets = gpt_result.find('{')
  close_curly_brackets = gpt_result.rfind('}')+1
  json_str = gpt_result[open_curly_brackets:close_curly_brackets]
  parsed = ast.literal_eval(json_str)
  if DEBUG:
    print('Resultado do GPT parseado:')
    print(parsed)
  return parsed['score']

def rerank_chunks(chunks_to_rerank, section_title):
  llm_gpt = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name=PARAMS['gpt_model_name_reranking'])

  chat_prompt = ChatPromptTemplate.from_messages([rerank_system_message_prompt,
                                                  rerank_human_message_prompt_task,
                                                  rerank_ai_message_prompt_yes,
                                                  rerank_human_message_prompt_text])
  question_chain = LLMChain(llm=llm_gpt, prompt=chat_prompt, verbose=True)

  scores_given_by_gpt = []

  for doc in chunks_to_rerank:
    text = get_prepared_text_from_document_chunk(doc)
    if DEBUG and len(text) > PARAMS["truncate_chunks_at_n_chars"]:
      print(f'Tamanho do chunk em caracteres: {len(text)}. Foi truncado em {PARAMS["truncate_chunks_at_n_chars"]} caracteres')
    gpt_result = question_chain.run(survey_topic=SURVEY_TOPIC_QUERY, section_title=section_title, text=text[0:PARAMS['truncate_chunks_at_n_chars']])
    if DEBUG:
      print('Resultado do GPT:')
      print(gpt_result)
    score = parse_result_to_get_score(gpt_result)
    scores_given_by_gpt.append(score)

  if DEBUG:
    print(f'Lista de scores do GPT para seção {section_title}:')
    print(scores_given_by_gpt)
  # Rerank
  reranked_chunks = [chunk for _, chunk in sorted(zip(scores_given_by_gpt, chunks_to_rerank), key=lambda pair: pair[0], reverse=True)]
  # Remove chunks with score 0
  reranked_chunks[0:len(scores_given_by_gpt)-scores_given_by_gpt.count(0)]

  return reranked_chunks

# Function to write the text of a section

In [ ]:
# SYSTEM MESSAGE
write_section_system_template = "You are a renowned scientist who is writing a survey entitled '{survey_topic}'."
write_section_system_message_prompt = SystemMessagePromptTemplate.from_template(write_section_system_template)

# FIRST HUMAN MESSAGE - EXPLAINING THE TASK
write_section_human_template_task = """\
Your task is to write the text of the section '{section_title}' of the survey. \
To complete this task, I will give you a list of documents that should be used as references. \
Each document has a text and an alphanumeric ID. \n\
When writing the section, you MUST follow this rules: \n\
- be aware of plagiarism, i.e., you should not copy the text, but use them as inspiration.\n\
- when using some reference, you must cite it right after its use. You should use the IEEE citing style (write the id of the text \
between square brackets).\n\
- you are writing the paragraphs of the section. You MUST write only this section.\n\
- you MUST NOT split the section in subsections, nor create introduction and conclusion for it.\n\
- DO NOT write any conclusion in any form for the subsection.\n\n\
- DO NOT write a references section.\n\
- DO NOT begin the text writing that the context is '{survey_topic}', as this is obvious from the title of the survey.\n\
Do you understand your task?\
"""
write_section_human_message_prompt_task = HumanMessagePromptTemplate.from_template(write_section_human_template_task)

# FIRST AI ANSWER - AGREEING
write_section_ai_message_prompt_yes = AIMessagePromptTemplate.from_template("Sure, send me a list of text and I will write a section about '{section_title}' using them as references. I am aware that I should use the IEEE citing style.")

# SECOND HUMAN MESSAGE - LIST OF DOCUMENTS
def write_section_text_message_human_prompt_list_documents(texts):
  message = ''
  for idx, text in enumerate(texts):
    if DEBUG and len(text) > PARAMS["truncate_chunks_at_n_chars"]:
      print(f'Tamanho do texto da referência: {len(text)}. Foi truncado em {PARAMS["truncate_chunks_at_n_chars"]} caracteres')
    message = message + f"ID: REF{idx}\nText: {text[0:PARAMS['truncate_chunks_at_n_chars']].replace('{', '{{').replace('}', '}}')}\n\n"
  return HumanMessagePromptTemplate.from_template(message)


In [ ]:
def convert_doc_references_to_primitive_type(references):
  result = []
  for ref in references:
    ref_in_simple_type = ref.metadata.copy()
    ref_in_simple_type['chunk'] = ref.page_content
    result.append(ref_in_simple_type)
  return result

def write_section(section_title, references_as_docs):
  llm_gpt = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name=PARAMS['gpt_model_name_write_text'])

  # Extract the first n_chunks_to_use_as_reference texts reranked by the GPT
  texts = []

  list_of_references = '\n\nReferences given to GPT: \n'
  debug_str_refs = ''
  for i, ref_as_doc in enumerate(references_as_docs[0:PARAMS['n_chunks_to_use_as_reference']]):
    text = get_prepared_text_from_document_chunk(ref_as_doc)
    texts.append(text)

    debug_str_refs = f'{debug_str_refs}\n\nREF{i}: {ref_as_doc.metadata["paperId"]} - {ref_as_doc.metadata["title"]}\n{ref_as_doc.page_content}\n'

  write_section_message_list_documents = write_section_text_message_human_prompt_list_documents(texts)

  # Now, generate the chat messages for this subsection
  chat_prompt = ChatPromptTemplate.from_messages([write_section_system_message_prompt,
                                                  write_section_human_message_prompt_task,
                                                  write_section_ai_message_prompt_yes,
                                                  write_section_message_list_documents])

  #chat_messages = chat_prompt.format_prompt(title_subsection=subsections_per_cluster[cluster]['title']).to_messages()
  #print(chat_messages)
  question_chain = LLMChain(llm=llm_gpt, prompt=chat_prompt, verbose=True)
  text = question_chain.run(survey_topic=SURVEY_TOPIC_QUERY, section_title=section_title)
  if DEBUG:
    print(section_title)
    print('\n')
    print(text)
    print('\nReferences sent to GPT:\n')
    print(debug_str_refs)

  return (text, convert_doc_references_to_primitive_type(references_as_docs[0:PARAMS['n_chunks_to_use_as_reference']]))


# Function to walk through sections and generate the text

In [ ]:
%%time

def get_full_section_title(previous_sections_titles, section_title):
  return ' - '.join(previous_sections_titles + [section_title])

def walk_sections_and_write_text(list_of_sections, previous_sections_titles = []):
  for section in list_of_sections:
    # Get the section title and the full section title (the last one is the concatenation between the previous and current titles)
    section_title = section['section_title']
    full_section_title = get_full_section_title(previous_sections_titles, section_title)

    # In case the code breaks, we can adjust it and rerun without generating the text that already have been generated
    if 'generated_text' in section:
      if DEBUG:
        print(f'The text for {full_section_title} have already been written. Skiping...')
      continue

    # Only write the the sections that does not have subsections
    if len(section['subsections']) == 0:
      if DEBUG:
        print(f'Writing the text for section {full_section_title}...')

      papers_in_section = section['papers']
      db = get_cached_vector_db(full_section_title, papers_in_section)
      related_chunks = search_in_vector_db(db, full_section_title, PARAMS['n_chunks_returned_by_vector_db'])
      reranked_chunks = rerank_chunks(related_chunks, full_section_title)
      text, references_sent_to_gpt = write_section(full_section_title, reranked_chunks)
      section['generated_text'] = {"text": text, "references_sent_to_gpt": references_sent_to_gpt}
    else:
      if DEBUG:
        print(f'The section {section_title} has subsections. Skip this section and write the text for its subsections...')
      # Walk through the subsections (and append the title of the current section)
      walk_sections_and_write_text(section['subsections'], previous_sections_titles + [section_title])


walk_sections_and_write_text(sections)

The section Text Representations for Ranking has subsections. Skip this section and write the text for its subsections...
Writing the text for section Text Representations for Ranking - BOW Encodings...
The database for section Text Representations for Ranking - BOW Encodings is not in cache. Creating...
{'paperId': 'BIBREF10_47ced790a563344efae66588b5fb7fe6cca29ed3', 'abstract': 'The Probabilistic Relevance Framework (PRF) is a formal framework for document retrieval, grounded in work done in the 1970—1980s, which led to the development of one of the most successful text-retrieval algorithms, BM25. In recent years, research in the PRF has yielded new retrieval models capable of taking into account document meta-data (especially structure and link-graph information). Again, this has led to one of the most successful Web-search and corporate-search algorithms, BM25F. This work presents the PRF from a conceptual point of view, describing the probabilistic modelling assumptions behind the

Splitting:   0%|          | 0/4 [00:00<?, ?it/s]

Indexing documents


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Retrieving database from cache


> Entering new  chain...
Prompt after formatting:
System: You are a renowned scientist who is writing the section 'Text Representations for Ranking - BOW Encodings' of a survey about 'neural information retrieval'.
Human: I've found a text excerpt from a scientifc article that might be useful for the section 'Text Representations for Ranking - BOW Encodings' of your survey about 'neural information retrieval'.
Your task is to generate a score for it ranging from 0 to 5 indicating its importance to the section that you are writing.
The score of a text written in a language other than English must be 0.
If the text contains only metadata of an article (title, authors, venue, etc), i.e. if the text belongs to the references session, the score should be 0.
You should also explain why you choose this score.
Your answer MUST be enclosed in a RFC8259 compliant JSON object with two properties, "score" and "reasoning", containing the score and the reasoning for 

...

Retrieving database from cache


> Entering new  chain...
Prompt after formatting:
System: You are a renowned scientist who is writing the section 'Learned Sparse Retrieval - Impact score learning' of a survey about 'neural information retrieval'.
Human: I've found a text excerpt from a scientifc article that might be useful for the section 'Learned Sparse Retrieval - Impact score learning' of your survey about 'neural information retrieval'.
Your task is to generate a score for it ranging from 0 to 5 indicating its importance to the section that you are writing.
The score of a text written in a language other than English must be 0.
If the text contains only metadata of an article (title, authors, venue, etc), i.e. if the text belongs to the references session, the score should be 0.
You should also explain why you choose this score.
Your answer MUST be enclosed in a RFC8259 compliant JSON object with two properties, "score" and "reasoning", containing the score and the reasoning for 

Splitting:   0%|          | 0/4 [00:00<?, ?it/s]

Indexing documents


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Retrieving database from cache


> Entering new  chain...
Prompt after formatting:
System: You are a renowned scientist who is writing the section 'Learned Sparse Retrieval - Sparse representation learning' of a survey about 'neural information retrieval'.
Human: I've found a text excerpt from a scientifc article that might be useful for the section 'Learned Sparse Retrieval - Sparse representation learning' of your survey about 'neural information retrieval'.
Your task is to generate a score for it ranging from 0 to 5 indicating its importance to the section that you are writing.
The score of a text written in a language other than English must be 0.
If the text contains only metadata of an article (title, authors, venue, etc), i.e. if the text belongs to the references session, the score should be 0.
You should also explain why you choose this score.
Your answer MUST be enclosed in a RFC8259 compliant JSON object with two properties, "score" and "reasoning", containing the score and 

In [ ]:
extraction_type = '_grobid' if 'grobid' in file_papers_contents else ''

with open(f'{file_sections_structure[0:-4]}{extraction_type}_generated_text_{PARAMS["embeddings"]}_{PARAMS["gpt_model_name_write_text"]}.pkl', 'wb') as f:
  pickle.dump(sections, f)

In [ ]:
def print_references(references, print_detailed_refs):
  for i, ref in enumerate(references):
    str_ref = f"REF{i} - {ref['title']}"
    print(str_ref)
    if (print_detailed_refs):
      print(ref['chunk'])

def print_survey(sections, print_detailed_refs=False, level=0, str_previous_section_numbers = ''):
  for i, section in enumerate(sections):
    str_sec_number = str_previous_section_numbers + ('' if level == 0 else '.') + str(i+1)
    section_title = '\t'*(level) + ' ' + str_sec_number + ' ' + section['section_title']
    print("\n" + section_title + "\n")

    if len(section['subsections']) > 0:
      print_survey(section['subsections'], print_detailed_refs, level+1, str_sec_number)
    else:
      print(section['generated_text']['text'])
      print("\nReferences sent to GTP: ")
      print_references(section['generated_text']['references_sent_to_gpt'], print_detailed_refs)

In [ ]:
print_survey(sections)


 1 Text Representations for Ranking


	 1.1 BOW Encodings

Text Representations for Ranking - BOW Encodings

Bag-of-Words (BOW) encodings are a popular approach for representing text documents in information retrieval systems. In this encoding scheme, each document is represented as a vector of term frequencies, where each dimension corresponds to a unique term in the document collection. BOW encodings have been widely used in traditional retrieval models, and their application in neural information retrieval has also gained attention [REF0].

The BOW encoding scheme assumes that the order of terms in a document does not convey important information for retrieval purposes. Instead, it focuses on the frequency of occurrence of terms within a document. This approach has been motivated by the observation that the presence and frequency of certain terms can be indicative of the document's relevance to a given query [REF1].

One advantage of BOW encodings is their simplicity and efficiency